In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

In [3]:
from fastai.text import data

Taken from here:  
https://gist.github.com/bearpelican/48cd4c505aea7c94e7c1e6e5e24bfac0

# Wikitext 103
This notebook is for training the language model on most of Wikipedia.  
The idea is to create a generalized language model before we fine tune it on a specialized task

## Data - Wikitext-103

Download the dataset [here](https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip) and unzip it so it's in the folder wikitext.

Blog:
https://einstein.ai/research/blog/the-wikitext-long-term-dependency-language-modeling-dataset

Original notebook:
https://github.com/fastai/fastai_docs/blob/master/dev_nb/007_wikitext_2.ipynb

Small helper function to read the tokens.

In [4]:
PATH=Path('data/wikitext-103')

In [5]:
# download_url('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip', PATH)

In [4]:
def read_file(fn:PathOrStr, enc='utf-8'):
    "Read the text in `fn`."
#     with open(fn,'r', encoding = enc) as f: return ''.join(f.read().splitlines())
    tokens = []
    with open(fn,'r', encoding = enc) as f: 
        for line in f.read().splitlines():
            l = line.strip()
            if len(l) == 0: continue
            tokens.append(l)#.split())
    return np.array(tokens)

In [6]:
train_tok = read_file(PATH/'wiki.train.tokens')
valid_tok = read_file(PATH/'wiki.valid.tokens')
test_tok = read_file(PATH/'wiki.test.tokens')

In [7]:
len(train_tok), len(valid_tok), len(test_tok)

(1165029, 2461, 2891)

In [8]:
train_tok[:2][:2]

array(['= Valkyria Chronicles III =',
       'Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .'],
      dtype='<U7061')

In [11]:
p = [#OpenLargeFileProcessor(), 
     TokenizeProcessor(chunksize=1000), 
     NumericalizeProcessor(vocab=None)]

In [17]:
a = TextList(train_tok, path=path, processor=p).label_for_lm()

In [ ]:
a.cre

In [19]:
a[0]

AttributeError: 'NoneType' object has no attribute 'textify'

In [18]:
db = a.databunch()

AttributeError: databunch

In [13]:
path = PATH
# processor = data._get_processor(tokenizer=tok, vocab=None, max_vocab=60000)
# processor = data._get_processor(vocab=None, chunksize=500)
src = ItemLists(path, TextList(train_tok, path=path, processor=p),
                TextList(valid_tok, path=path, processor=p))
src = src.label_for_lm()
if test_tok is not None: src.add_test(TextList(test_tok, path=path))
text_data = src.databunch()

OSError: [Errno 12] Cannot allocate memory

In [14]:
%debug

> /home/ubuntu/anaconda3/envs/fastai/lib/python3.7/multiprocessing/popen_fork.py(70)_launch()
     68         code = 1
     69         parent_r, child_w = os.pipe()
---> 70         self.pid = os.fork()
     71         if self.pid == 0:
     72             try:

ipdb> up
> /home/ubuntu/anaconda3/envs/fastai/lib/python3.7/multiprocessing/popen_fork.py(20)__init__()
     18         self.returncode = None
     19         self.finalizer = None
---> 20         self._launch(process_obj)
     21 
     22     def duplicate_for_child(self, fd):

ipdb> up
> /home/ubuntu/anaconda3/envs/fastai/lib/python3.7/multiprocessing/context.py(277)_Popen()
    275         def _Popen(process_obj):
    276             from .popen_fork import Popen
--> 277             return Popen(process_obj)
    278 
    279     class SpawnProcess(process.BaseProcess):

ipdb> up
> /home/ubuntu/anaconda3/envs/fastai/lib/python3.7/multiprocessing/process.py(112)start()
    110                'daemonic processes are not allowed 

ipdb> len(texts)
1000
ipdb> self.n_cpus
8
ipdb> quit


In [ ]:
# text_data = TextLMDataBunch.from_tokens(
#     PATH, train_tok, valid_tok, test_tok, None, None)

In [ ]:
vocab = text_data.train_ds.vocab

In [ ]:
text_data.save()

### Wikitext-2

In [5]:
PATH=Path('data/wikitext-2')

In [6]:
# defaults.text_spec_tok = [PAD,UNK,BOS,FLD,TK_MAJ,TK_UP,TK_REP,TK_WREP]

In [6]:
# download_url('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip', PATH)

In [7]:
train_tok = read_file(PATH/'wiki.train.tokens')
valid_tok = read_file(PATH/'wiki.valid.tokens')
test_tok = read_file(PATH/'wiki.test.tokens')

In [8]:
train_tok[:2]

array(['= Valkyria Chronicles III =',
       'Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .'],
      dtype='<U3834')

In [9]:
len(train_tok), len(valid_tok), len(test_tok)

(23767, 2461, 2891)

In [10]:
' '.join(train_tok[4][:4])

'=   =  '

In [11]:
test = data._join_texts(train_tok)

In [8]:
path = PATH
processor = data._get_processor(vocab=None)
src = ItemLists(path, TextList(train_tok, path=path, processor=processor),
                TextList(valid_tok, path=path, processor=processor))
src = src.label_for_lm()
if test_tok is not None: src.add_test(TextList(test_tok, path=path))
text_data = src.databunch()

In [13]:
# text_data = TextLMDataBunch.from_tokens(
#     PATH, train_tok, valid_tok, test_tok, None, None)

In [9]:
text_data.save('tmp_rev')

In [11]:
text_data.show_batch()

idx,text
0,xxbos = = xxmaj charitable and feminist activities = = xxbos = = = xxmaj sovetskaya xxmaj belorussiya = = = xxbos ( a ) xxunk sort of bird called solitaires which are very good
1,"the xxunk scar is a circular xxunk at one end of the spore , and it most likely results during the separation of the attachment of the spore to the xxunk of the xxunk ."
2,"around him . xxmaj mitch xxmaj pileggi had to endure long bouts of make - up application , a process that he admitted he "" hated "" . xxmaj the nanobots in the blood sample"
3,". xxmaj the "" xxmaj battle of the xxunk "" involved a demonstration against a lack of access to land and the serving of xxunk notices . xxmaj the incident involved numerous xxunk and"
4,"followers into small groups , which they referred to as "" xxmaj star xxunk "" , xxbos 3 , and anions such as xef − xxbos xxmaj oldham 's xxunk was dominated by distinctive rectangular"


In [13]:
text_data.vocab.stoi[PAD]

0

## Loading data

In [15]:
text_data = TextLMDataBunch.load(PATH)

In [15]:
text_data.show_batch()

idx,text
0,"xxbos = = xxmaj applications = = xxbos xxmaj the episode received mixed reception in xxmaj australia , with some xxmaj australian fans saying the episode was a mockery of their country . xxmaj shortly after it had aired , the xxmaj simpsons staff received over 100 letters from xxmaj australians who were insulted by the episode . xxmaj they also received letters from people complaining about the xxmaj australian"
1,"pair of shoes "" . xxmaj the video was released on 30 xxmaj november 2011 . xxmaj after the official release , xxmaj nadia xxmaj mendoza of the xxmaj daily xxmaj mail stated the video was a departure from her previous two videos ( for "" xxmaj beat of xxmaj my xxmaj drum "" and "" xxmaj lucky xxmaj day "" ) , saying that it "" looks and sounds"
2,"among the top three in regular - season xxup mvp voting a record 10 times , and was named one of the 50 xxmaj greatest xxmaj players in xxup nba xxmaj history in 1996 . xxmaj he is one of only seven players in history to win an xxup ncaa championship , an xxup nba championship , and an xxmaj olympic gold medal ( doing so twice with the 1984"
3,". "" xxmaj she ended with : "" xxmaj it 's xxunk . xxunk no one should have to think that hard to keep up with the xxmaj joes . "" xxmaj another negative review came from xxmaj variety xxmaj magazine 's xxmaj justin xxmaj chang , who ridiculed the movie 's large - scale destruction of foreign cities , writing : "" xxunk who thrilled to the sight of"
4,"employed for brief periods at radio station xxunk , and became active in the local chapter of the xxunk xxunk xxmaj association of xxmaj america . xxmaj from 1979 he worked at xxmaj national xxmaj public xxmaj radio - affiliate ( xxup npr ) xxunk until 1981 when he was asked to submit his resignation after a dispute about the requirements of objective focus in his presentation of news ."
